In [ ]:
import pandas as pd
import numpy as np
import nltk 
from bs4 import BeautifulSoup            
import re
import time
import ast
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import Counter
import math
from textblob import TextBlob as tb
from sklearn.pipeline import Pipeline


In [ ]:
nameTrainCSV = 'trainWithListOfCleanWords'
nameTestCSV = 'testWithListOfCleanWords'

train = pd.read_csv('../data/processed/' + nameTrainCSV + '.csv', encoding='utf-8')
train['BagOfWords'] = dict
train.head()


In [ ]:
start = time.time()
for x in range(len(train)):
    train.set_value(col='listOfCleanWords',
                index=x,
                value=ast.literal_eval(train["listOfCleanWords"][x]))
    train.set_value(col='BagOfWords',
                index=x,
                value=Counter(train["listOfCleanWords"][x]))
end = time.time()
print(end - start)

In [ ]:
train.head()

In [ ]:
def getDataFrameFilterByToxicClass(df, toxicName, cls = 1):
    return train[train[toxicName] == cls]['listOfCleanWords']

toxicName = 'toxic'

clnTxtWrds = getDataFrameFilterByToxicClass(train, toxicName).tolist()
clnTxtWrdsNoToxic = getDataFrameFilterByToxicClass(train, toxicName, 0).tolist()

allTextsToxic = [' '.join(clnTxtWrds[x]) for x in range(len(clnTxtWrds))]
allTextsNoToxic = [' '.join(clnTxtWrdsNoToxic[x]) for x in range(len(clnTxtWrdsNoToxic))]

In [ ]:
allTextsToxic[:10]

In [ ]:
allTextsNoToxic[:10]

In [ ]:
topNFeatures = 500
tdidfVectTrainToxic = TfidfVectorizer(max_features=topNFeatures)
matrixTdidfTrainToxic = tdidfVectTrainToxic.fit_transform(allTextsToxic)

featureArrayToxicTrain = np.array(tdidfVectTrainToxic.get_feature_names())
tfidfOrderedToxicTrain = np.argsort(matrixTdidfTrainToxic.toarray()).flatten()[::-1]


tdidfVectTrainNoToxic = TfidfVectorizer(max_features=topNFeatures)
matrixTdidfTrainNoToxic = tdidfVectTrainNoToxic.fit_transform(allTextsNoToxic)

featureArrayNoToxicTrain = np.array(tdidfVectTrainNoToxic.get_feature_names())
tfidfOrderedNoToxicTrain = np.argsort(matrixTdidfTrainNoToxic.toarray()).flatten()[::-1]




In [ ]:
selectedWordsTrainToxic = featureArrayToxicTrain[tfidfOrderedToxicTrain][:topNFeatures]
selectedWordsTrainNoToxic = featureArrayNoToxicTrain[tfidfOrderedNoToxicTrain][:topNFeatures]
selectedWordsTrain = []
for x in range(topNFeatures):
    if len(selectedWordsTrain) <= topNFeatures: 
        selectedWordsTrain = np.unique(np.append(selectedWordsTrain, np.unique([selectedWordsTrainNoToxic[x], selectedWordsTrainToxic[x]])))    
selectedWordsTrain = selectedWordsTrain[:topNFeatures]


In [ ]:
len(selectedWordsTrain)

In [ ]:
# With all selected toxic words get BoW from all Texts 

clnTxtWrds = train['listOfCleanWords']
allTexts = [' '.join(clnTxtWrds[x]) for x in range(len(clnTxtWrds))]

vectTrain = CountVectorizer(vocabulary=selectedWordsTrain)
matrixBoWTrain = vectTrain.fit_transform(allTexts).toarray()

In [ ]:
matrixBoWTrain.shape

In [ ]:
matrixBoWTrain[6]

In [ ]:
np.where(matrixBoWTrain[6] == 1)

In [ ]:
selectedWordsTrain[:10]

In [ ]:
[selectedWordsTrain[x] for x in np.where(matrixBoWTrain[6] == 1)[0]]

In [ ]:
clnTxtWrds[6]

In [ ]:
print(matrixBoWTrain)

In [ ]:
test = pd.read_csv('../data/processed/' + nameTestCSV + '.csv', encoding='utf-8')
test.head()

In [ ]:
clnTxtWrdsTest = test['listOfCleanWords'].tolist()
allTextsTest = [' '.join(clnTxtWrdsTest[x]) for x in range(len(clnTxtWrdsTest))]

In [ ]:
vectTest = CountVectorizer(vocabulary=selectedWordsTrain)
matrixBoWTest = vectTest.fit_transform(allTextsTest).toarray()

In [ ]:
matrixBoWTest.shape

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(matrixBoWTrain, train['toxic'].tolist())

In [ ]:
predicted = clf.predict(matrixBoWTest)

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer(vocabulary=selectedWordsTrain)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
                        ])
text_clf = text_clf.fit(allTexts, train['toxic'].tolist())

In [ ]:
predicted2 = text_clf.predict(allTextsTest)


In [ ]:
np.where(predicted > 0)

In [ ]:
np.where(predicted2 == 1)

In [ ]:
train['toxic']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 10) 
forest = forest.fit( matrixBoWTrain, train["toxic"].tolist())


In [ ]:
result = forest.predict(matrixBoWTest)

In [ ]:
result

In [ ]:
np.where(result == 1)